In [1]:
import numpy as np
import os
import cv2

In [2]:
#Functia va returna images, labels

#pt training directory, unde structura este de forma asl_alphabet_train/asl_alphabet_train/{label}/{image}
def load_images_train(directory):
    images = []
    labels = []
    for idx, label in enumerate(unique_labels):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(cv2.imread(filepath), (64, 64))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

#pt testing directory, unde structura este de forma asl_alphabet_test/asl_alphabet_test/{label}_'test.jpg'
def load_images_test(directory):
    images = []
    labels = []
    for idx, label in enumerate(unique_labels):
        print(directory + "/" + label + "_test")
        image = cv2.resize(cv2.imread(directory + "/" + label + "_test.jpg"), (64, 64))
        images.append(image)
        labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

In [3]:
import keras
train_dir = 'asl_alphabet_train/asl_alphabet_train'
eval_dir = 'asl_alphabet_test/asl_alphabet_test'
unique_labels = sorted(os.listdir(train_dir))
print(unique_labels)
images, labels = load_images_train(directory = train_dir)
img_eval, lbl_eval = load_images_test(directory = eval_dir)


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'nothing', 'space']
asl_alphabet_test/asl_alphabet_test/A_test
asl_alphabet_test/asl_alphabet_test/B_test
asl_alphabet_test/asl_alphabet_test/C_test
asl_alphabet_test/asl_alphabet_test/D_test
asl_alphabet_test/asl_alphabet_test/E_test
asl_alphabet_test/asl_alphabet_test/F_test
asl_alphabet_test/asl_alphabet_test/G_test
asl_alphabet_test/asl_alphabet_test/H_test
asl_alphabet_test/asl_alphabet_test/I_test
asl_alphabet_test/asl_alphabet_test/J_test
asl_alphabet_test/asl_alphabet_test/K_test
asl_alphabet_test/asl_alphabet_test/L_test
asl_alphabet_test/asl_alphabet_test/M_test
asl_alphabet_test/asl_alphabet_test/N_test
asl_alphabet_test/asl_alphabet_test/O_test
asl_alphabet_test/asl_alphabet_test/P_test
asl_alphabet_test/asl_alphabet_test/Q_test
asl_alphabet_test/asl_alphabet_test/R_test
asl_alphabet_test/asl_alphabet_test/S_test
asl_alphabet_test/asl_alphabet_t

In [4]:
from sklearn.model_selection import train_test_split

#train data & test data
img_train, img_test, lbl_train, lbl_test = train_test_split(images, labels, test_size = 0.1, stratify = labels)

print("Number of labels: ", len(unique_labels))
print("Training img number: " , len(img_train))
print("Testing img number: ", len(img_test))
print("Eval img number:", len(img_eval))

Number of labels:  28
Training img number:  75600
Testing img number:  8400
Eval img number: 28


In [5]:
#one hot enconding using to_categorical.
import tensorflow as tf
from keras import utils as np_utils

lbl_train = tf.keras.utils.to_categorical(lbl_train)
lbl_test = tf.keras.utils.to_categorical(lbl_test)
lbl_eval = tf.keras.utils.to_categorical(lbl_eval)
#print(lbl_train[1000])
print(len(lbl_train[0]))

28


In [6]:
#data nnorm
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0
img_eval = img_eval.astype('float32')/255.0
print(img_train[0])

[[[0.8980392  0.00392157 0.00784314]
  [0.74509805 0.04313726 0.05098039]
  [0.7372549  0.04705882 0.04705882]
  ...
  [0.7372549  0.04313726 0.04313726]
  [0.7529412  0.03529412 0.05490196]
  [0.84313726 0.04705882 0.0627451 ]]

 [[0.7490196  0.05098039 0.04705882]
  [0.4117647  0.43137255 0.38431373]
  [0.40784314 0.42745098 0.39607844]
  ...
  [0.18431373 0.1882353  0.20784314]
  [0.23921569 0.1764706  0.21960784]
  [0.49019608 0.17254902 0.19607843]]

 [[0.75686276 0.06666667 0.04313726]
  [0.5921569  0.6117647  0.5294118 ]
  [0.5803922  0.5882353  0.5019608 ]
  ...
  [0.23529412 0.24313726 0.28235295]
  [0.24705882 0.23529412 0.2784314 ]
  [0.5176471  0.2        0.22745098]]

 ...

 [[0.74509805 0.02352941 0.00784314]
  [0.14117648 0.08235294 0.0627451 ]
  [0.16078432 0.09803922 0.09411765]
  ...
  [0.09411765 0.19607843 0.3137255 ]
  [0.11372549 0.18431373 0.29803923]
  [0.42745098 0.16862746 0.25490198]]

 [[0.7647059  0.04313726 0.03137255]
  [0.16470589 0.11372549 0.10980392]


In [10]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', 
                 input_shape = (64, 64, 3)))
model.add(MaxPooling2D(pool_size = (4, 4)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(28, activation='softmax'))
model.summary()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy',tf.keras.metrics.Recall()])
hist = model.fit(img_train, lbl_train, epochs = 2, batch_size = 100)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                458780    
Total params: 463,644
Trainable params: 463,644
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
756/756 [==============================] - 170s 223ms/step - loss: 1.4987 - accuracy: 0.5688 - recall_2:

In [11]:

model2 = Sequential()
model2.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', 
                 input_shape = (64, 64, 3)))
model2.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu'))
model2.add(MaxPooling2D(pool_size = (4, 4)))
model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(28, activation='softmax'))
model2.summary()
model2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy',tf.keras.metrics.Recall()])
hist2 = model2.fit(img_train, lbl_train, epochs = 2, batch_size = 100)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 28)                458780    
Total params: 566,108
Trainable params: 566,108
Non-trainable params: 0
________________________________________________

In [13]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential

model3 = Sequential()
model3.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', 
                 input_shape = (64, 64, 3)))
model3.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (4, 4)))
model3.add(Dropout(0.5))
model3.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (4, 4)))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(28, activation='softmax'))

model3.summary()

model3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy',tf.keras.metrics.Recall()])
hist = model3.fit(img_train, lbl_train, epochs = 2, batch_size = 100)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 128)       409728    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 128)        

In [15]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential

model3 = Sequential()
model3.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', 
                 input_shape = (64, 64, 3)))
model3.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (4, 4)))
model3.add(Dropout(0.5))
model3.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (4, 4)))
model3.add(Dropout(0.5))
model3.add(Flatten())
model3.add(Dense(28, activation='softmax'))

model3.summary()

model3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy',tf.keras.metrics.Recall()])
hist = model3.fit(img_train, lbl_train, epochs = 5, batch_size = 100)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 64)        4864      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 64, 64)        102464    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 16, 16, 128)       204928    
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 16, 128)       409728    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 4, 4, 128)        

In [12]:
model.save('md1.h5')
model2.save('md2.h5')
#score = model.evaluate(x = img_test[:50], y = lbl_test[:50], verbose = 0)
#print('Accuracy for test images:', round(score[1]*100, 3), '%')

#score = model.evaluate(x = img_eval, y = lbl_eval, verbose = 0)
#print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')


In [9]:
import cv2
import numpy as np
import tensorflow as tf
model = tf.keras.models.load_model('md.h5')
cap = cv2.VideoCapture(0)
unique_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'nothing', 'space']
while(True):
      
    # Capture frames in the video
    ret, frame = cap.read()


    ################################
    #frame = cv2.flip(frame,1)
  
    #define roi
    roi = frame[100:400, 320:620]
    cv2.imshow('roi',roi)

    # describe the type of font
    # to be used.
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Use putText() method for
    # inserting text on video
    cv2.rectangle(frame, (320, 100), (620, 400), (0, 0, 0), 0)
    image = cv2.resize(roi, (64, 64))
    to_send = np.array(image).astype('float32')/255.0
    to_send = to_send.reshape(-1, 64, 64, 3)
    string = unique_labels[np.argmax(model.predict(to_send), axis=1)[0]]
    print(string)
    cv2.putText(frame, 
                string, 
                (50, 50), 
                font, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)
     # Display the resulting frame
    cv2.imshow('video', frame)
    # creating 'q' as the quit 
    # button for the video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# release the cap object
cap.release()
# close all windows
cv2.destroyAllWindows()

Y
Y
Y
Y
Y
P
Y
P
P
P
P
Y
P
P
P
T
P
P
Y
Y
Y
T
T
T
T
T
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
